# <font color="AA0088">Exam Scores</font>

## <font color="AA0088">Project Participants</font>

Stakeholder(s): Me

Project Manager(s): Me

Data Analyst(s): erm.... Me

## <font color="AA0088">Problem Definition</font>

The purpose of this project is to gather insights from a collection of exam scores to understand the following main goals:

* The average score for each gender
* The average score for each group

## <font color="AA0088">Import Required Libraries</font>

The below libraries are required by this notebook.

In [3]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sn
import sklearn as skl

Show the versions of the libraries that will be used.

In [4]:
print(f"Matplotlib Version:   {mpl.__version__}")
print(f"Numpy Version:        {np.__version__}")
print(f"Pandas Version:       {pd.__version__}")
print(f"Seaborn Version:      {sn.__version__}")
print(f"SKLearn Version:      {skl.__version__}")

Matplotlib Version:   3.5.3
Numpy Version:        1.23.3
Pandas Version:       1.4.4
Seaborn Version:      0.12.0
SKLearn Version:      1.1.3


## <font color="AA0088">Data Processing</font>

### Data Source(s)

The data for this project was acquired from the following location(s):

### Data Overview

Give an overview of what the data represents as a whole.

### Data Dictionary

No data dictionary has been provided by the supplier of the data. From initial investigation, the features of the data include:

* gender: string: The gender of the examinees
* race/ethnicity: string: The race / ethnicity of the examinees. Each race / ethnicity is described as a group for anonymity
* parental level of education: string: The highest level of education of an examinees parent
* lunch: string: The category of lunch an examinee has
* test preparation course: string: The completion status for the test preparation course for an examinee
* math score: int: The examinees score for their maths test
* reading score: int: The examinees score for their reading test
* writing score: int: The examinees score for their writing test

### Import The Data

First step is to import the data to a pandas dataframe from the source of the data. In this case, the source data will be a csv file.

In [5]:
raw_data_df = pd.read_csv("./data/raw_data.csv")

Let us have a quick look at the first five rows of the dataframe.

In [6]:
#raw_data_df.head(n = 5)
raw_data_df.info()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,male,group A,high school,standard,completed,67,67,63
1,female,group D,some high school,free/reduced,none,40,59,55
2,male,group E,some college,free/reduced,none,59,60,50
3,male,group B,high school,standard,none,77,78,68
4,male,group E,associate's degree,standard,completed,78,73,68


### Checking The Data

### Cleaning Up The Data

## <font color="AA0088">Exploratory Data Analysis</font>

## <font color="AA0088">Data Classification and Modelling</font>

### Cross-Valaidation

### Hyper-Parameter Tuning

## <font color="AA0088">Evaluation</font>

## <font color="AA0088">Features</font>